---
title: TUI
output-file: tui.html
---

## TUI Module - Terminal User Interface

This module implements the complete Textual-based terminal user interface for live audio transcription with AI-powered editing capabilities.

### Overview

The TUI provides a full-featured interactive interface that combines: <br>
- Real-time speech transcription with Faster Whisper <br>
- AI-powered edit detection and application <br>
- Multi-provider AI support (OpenAI, Anthropic, Google) <br>
- Keyboard-driven workflow with modal dialogs <br>
- Visual feedback for recording state and AI operations <br>

### Architecture

The interface is built with Textual, a modern Python framework for building terminal UIs. The architecture consists of:

1. **Main Application** (`TranscriptionTUI`): Coordinates all components and manages state <br>
2. **Modal Screens**: Settings dialogs for audio and AI configuration <br>
3. **Help System**: Comprehensive in-app documentation <br>
4. **Event Handlers**: Keyboard shortcuts and user interactions <br>
5. **Reactive UI**: Automatic updates based on state changes <br>

### Key Features

- **Live Transcription Display**: Scrolling log that shows transcripts in real-time <br>
- **Recording Indicator**: Header turns red during active recording <br>
- **Status Notifications**: Toast-style messages for user feedback <br>
- **Clipboard Integration**: One-key copy of entire transcript <br>
- **Persistent Configuration**: Remembers settings and API keys between sessions <br>

In [ ]:
#| default_exp tui


In [ ]:
#| export
from enum import Enum, auto
import asyncio
import pyperclip
import os

from textual import on
from textual.app import App, ComposeResult
from textual.reactive import reactive
from textual.containers import Container, Grid, Center, HorizontalGroup
from textual.screen import ModalScreen
from textual.widgets import Header, Footer, Static, Button, Log, Select, Rule, MarkdownViewer, Input, Switch

from tui_writer.live import LiveTranscriber
from tui_writer.ai import TranscriptEditor
from tui_writer.config import get_cfg

## Styling with Textual CSS

The CSS defines the visual appearance of the TUI using Textual's CSS-like syntax. Key features: <br>
- **Recording indicator**: Changes header background to red (`$error`) when recording <br>
- **Transcript display**: Styled with hatching pattern and padding for readability <br>
- **Modal layouts**: Centered dialogs for settings and help screens <br>
- Uses Textual's built-in color variables like `$background`, `$surface`, `$boost`, etc. <br>

In [ ]:
#| export

TEXTUAL_CSS = """
    Screen {
        align: center top;
    }
    Header {
        background: $background;
    }
    .recording {
        background: $error;
    }

    #transcript-display {
        hatch: horizontal $boost 80%;
        background: $boost;
        margin: 1 3;
        padding: 2 3;
    }
    
    HelpModal {
        align: center middle;
    }
    #help {
        border: thick $background 80%;
        background: $surface;
        width: 100;
        padding: 0 2;
        align: center top;
        height: 90%;
        max-height: 70;
    }

    
    SettingsModal {
        align: center middle;
    }
    #settings-modal {
        border: thick $background 80%;
        background: $surface;
        width: 100;
        padding: 0 2;
        align: center top;
        height: 25;
    }
    
    AISettingsModal {
        align: center middle;
    }
    #ai-settings-modal {
        border: thick $background 80%;
        background: $surface;
        width: 120;
        padding: 0 2;
        align: center top;
        height: 40;
    }
    
    #modal-header {
        content-align: center middle;
        margin: 1 0;
    }
    Rule {
        margin: 0;
    }
    #settings-grid {
        layout: grid;
        grid-size: 3 4;
        grid-rows: 1fr;
        grid-columns: 1fr;
        grid-gutter: 1;
    }
    #ai-settings-grid {
        layout: grid;
        grid-size: 3;
        grid-rows: 1fr;
        grid-columns: 1fr;
        grid-gutter: 1;
    }

    .select-text {
        height: 100%;
        content-align: center middle;
    }
    Select {
        column-span: 2;
        height: 100%;
        align: center middle;
    }
    Switch {
        column-span: 2;
        align: center middle;
    }
    Input {
        column-span: 2;
        height: 100%;
        align: center middle;
    }
    #settings-actions {
        column-span: 3;
    }
    #ai-settings-actions {
        width: auto;
    }
"""

## SettingsModal - Audio Configuration

Modal dialog for configuring Whisper transcription settings. Accessed by pressing `s` in the main interface.

### Available Models

The modal offers 10 Whisper model options with trade-offs between speed and accuracy:

**Standard Models** (multilingual): <br>
- **Tiny**: Ultra fast, low accuracy - Good for testing or real-time needs <br>
- **Base**: Fast, decent accuracy - Recommended starting point <br>
- **Small**: Balanced speed and accuracy - Good general purpose choice <br>
- **Medium**: Slow, high accuracy - For when quality matters <br>
- **Large-v3**: Very slow, best accuracy - Maximum quality <br>

**English-Only Models** (faster variants): <br>
- **Tiny.en** through **Large.en**: Optimized for English, typically 2x faster than multilingual equivalents <br>

### Language Support

Supports 40+ languages including: <br>
- European: English, German, French, Spanish, Italian, Portuguese, Dutch, Polish, Czech, Slovak, etc. <br>
- Nordic: Norwegian, Swedish, Danish, Finnish <br>
- Slavic: Russian, Ukrainian, Serbian, Croatian, Bulgarian, Slovenian <br>
- Asian: Chinese (Mandarin/Cantonese), Japanese, Korean, Hindi, Bengali, Thai, Vietnamese, Indonesian <br>
- Middle Eastern: Arabic, Hebrew, Persian, Turkish, Urdu <br>
- African: Swahili, Afrikaans <br>

### Implementation

Uses Textual's `Select` widgets for dropdown menus. Selected values are returned when the modal is dismissed and applied via the `apply_settings()` callback in the main app.

In [ ]:
#| export
#| include: false

class SettingsModal(ModalScreen):
    # Available Whisper model options (label, value)
    WHISPER_MODELS = [
        ("Tiny      ⚡  Ultra fast   - Low accuracy", "tiny"),
        ("Base      ⚡  Fast         - Decent accuracy", "base"),
        ("Small     ⚖   Balanced    - Good accuracy", "small"),
        ("Medium    🐢  Slow        - High accuracy",  "medium"),
        ("Large     🐢  Very Slow   - Best accuracy", "large-v3"),
        ("Tiny.en   English-only    - faster", "tiny.en"),
        ("Base.en   English-only    - faster", "base.en"),
        ("Small.en  English-only    - faster", "small.en"),
        ("Medium.en English-only    - faster", "medium.en"),
        ("Large.en  English-only    - faster", "large-v3.en")
    ]
    WHISPER_LANGUAGES = [
        ("English", "en"),
        ("Norwegian", "no"),
        ("Swedish", "sv"),
        ("Danish", "da"),
        ("Finnish", "fi"),
        ("German", "de"),
        ("Dutch", "nl"),
        ("French", "fr"),
        ("Spanish", "es"),
        ("Portuguese", "pt"),
        ("Italian", "it"),
        ("Polish", "pl"),
        ("Czech", "cs"),
        ("Slovak", "sk"),
        ("Hungarian", "hu"),
        ("Greek", "el"),
        ("Turkish", "tr"),
        ("Russian", "ru"),
        ("Ukrainian", "uk"),
        ("Arabic", "ar"),
        ("Hebrew", "he"),
        ("Hindi", "hi"),
        ("Bengali", "bn"),
        ("Urdu", "ur"),
        ("Persian (Farsi)", "fa"),
        ("Thai", "th"),
        ("Vietnamese", "vi"),
        ("Indonesian", "id"),
        ("Malay", "ms"),
        ("Filipino (Tagalog)", "tl"),
        ("Chinese (Mandarin)", "zh"),
        ("Chinese (Cantonese)", "yue"),
        ("Japanese", "ja"),
        ("Korean", "ko"),
        ("Swahili", "sw"),
        ("Afrikaans", "af"),
        ("Romanian", "ro"),
        ("Bulgarian", "bg"),
        ("Serbian", "sr"),
        ("Croatian", "hr"),
        ("Slovenian", "sl"),
        ("Estonian", "et"),
        ("Latvian", "lv"),
        ("Lithuanian", "lt"),
    ]

    def __init__(self, model, language, transcribe_only):
        self.model = model
        self.language = language
        self.transcribe_only = transcribe_only
        super().__init__()

    def compose(self) -> ComposeResult:
        with Container(id="settings-modal"):
            yield Static("SETTINGS", id="modal-header")
            yield Rule(line_style="heavy")
            with Grid(id="settings-grid"):
                yield Static("Transcribing model:", classes="select-text")
                yield Select(
                        options=self.WHISPER_MODELS,
                        allow_blank=False,
                        value=self.model,
                        id="whisper-model"
                )
                yield Static("Language:", classes="select-text")
                yield Select(
                        options=self.WHISPER_LANGUAGES,
                        allow_blank=False,
                        value=self.language,
                        id="whisper-language"
                )
                yield Static("Transcribe only mode:", classes="select-text")
                yield Switch(value=self.transcribe_only,animate=False, id="transcribe_only")
                yield Center(
                    Button.success("Close (esc)", id="save"),
                    id="settings-actions",
                )


    # Whisper selectors
    @on(Select.Changed, "#whisper-model")
    def on_model_changed(self, event: Select.Changed) -> None:
        self.model = str(event.value)

    @on(Select.Changed, "#whisper-language")
    def on_language_changed(self, event: Select.Changed) -> None:
        self.language = str(event.value)

    @on(Switch.Changed, "#transcribe_only")
    def on_transcribe_only_changed(self, event: Switch.Changed) -> None:
        self.transcribe_only = event.value

    def key_escape(self) -> None:
        self.dismiss([self.model, self.language, self.transcribe_only])

    def on_button_pressed(self) -> None:
        self.dismiss([self.model, self.language, self.transcribe_only])


## AI Settings Modal

Advanced modal for managing AI models across different providers. Accessed by pressing `a` in the main interface.

### Multi-Provider Architecture

The modal provides a unified interface for three LLM providers:

**OpenAI** - Most popular, wide range of models: <br>
- GPT-4o: Latest flagship model, fast and capable <br>
- GPT-4o Mini: Smaller, faster, cost-effective variant <br>
- GPT-4 Turbo: Previous generation flagship <br>
- GPT-4: Original GPT-4 <br>
- GPT-3.5 Turbo: Fast and economical <br>

**Anthropic** - Claude models known for safety and helpfulness: <br>
- Claude 3.5 Sonnet: Latest and most capable <br>
- Claude 3 Opus: Powerful reasoning and analysis <br>
- Claude 3 Sonnet: Balanced performance <br>
- Claude 3 Haiku: Fast and efficient <br>

**Google Gemini** - Google's multimodal models: <br>
- Gemini 1.5 Pro: Large context window, powerful <br>
- Gemini 1.5 Flash: Fast inference, economical <br>
- Gemini 1.0 Pro: Previous generation <br>

### Dynamic UI Updates

When you change providers, the modal: <br>
1. Updates the model dropdown with provider-specific options <br>
2. Loads any existing API key for that provider (displayed as masked dots) <br>
3. Updates the placeholder text in the API key field <br>
4. Preserves the last selected model if switching back to a previous provider <br>

### API Key Management

API keys are: <br>
- Stored in XDG-compliant config file at `~/.config/tui_writer/tui_writer.conf` <br>
- Displayed as dots for security when present <br>
- Only updated if you type a new value (leaving blank keeps existing key) <br>
- Loaded into environment variables (`OPENAI_API_KEY`, etc.) when the app starts <br>

### Persistence

The modal remembers: <br>
- Last used provider <br>
- Last selected model for each provider <br>
- API keys for all providers <br>

This allows seamless switching between providers without re-entering credentials.

In [ ]:
#| export
#| include: false

PROVIDERS = {
    "openai": {
        "label": "OpenAI",
        "cfg_key": "openai_key",
        "models": [
            ("GPT-4o", "gpt-4o"),
            ("GPT-4o Mini", "gpt-4o-mini"),
            ("GPT-4 Turbo", "gpt-4-turbo"),
            ("GPT-4", "gpt-4"),
            ("GPT-3.5 Turbo", "gpt-3.5-turbo"),
        ],
    },
    "anthropic": {
        "label": "Anthropic",
        "cfg_key": "anthropic_key",
        "models": [
            ("Claude 3.5 Sonnet", "claude-3-5-sonnet-20241022"),
            ("Claude 3 Opus", "claude-3-opus-20240229"),
            ("Claude 3 Sonnet", "claude-3-sonnet-20240229"),
            ("Claude 3 Haiku", "claude-3-haiku-20240307"),
        ],
    },
    "gemini": {
        "label": "Google Gemini",
        "cfg_key": "gemini_key",
        "models": [
            ("Gemini 2.5 Pro", "gemini-2.5-pro"),
            ("Gemini 2.5 Flash", "gemini-2.5-flash"),
        ],
    },
}

def provider_options():
    """[(label, value)] for Select."""
    return [(v["label"], k) for k, v in PROVIDERS.items()]

def model_options_for(provider: str):
    return PROVIDERS.get(provider, {}).get("models", [])

def cfg_key_for(provider: str):
    return PROVIDERS.get(provider, {}).get("cfg_key", "")

def mask_key(key: str) -> str:
    return "•" * 8 if key else ""


class AISettingsModal(ModalScreen):
    """Modal for managing AI model settings across multiple providers."""

    def __init__(self):
        super().__init__()
        self.cfg = get_cfg()

        # Resolve defaults from cfg if not provided
        self.provider = self.cfg.get("last_provider") or "openai"
        self.current_model = self.cfg.get("last_model") or ""
    
        # track whether there is an existing key (to keep if input left blank)
        self._existing_key = self._read_key(self.provider)

    def compose(self) -> ComposeResult:
        with Container(id="ai-settings-modal"):
            yield Static("AI MODEL SETTINGS", id="modal-header")
            yield Rule(line_style="heavy")
            
            with Grid(id="ai-settings-grid"):
                # Provider selection
                yield Static("AI Provider:", classes="select-text")
                yield Select(
                    options=provider_options(),
                    allow_blank=False,
                    value=self.provider,
                    id="ai-provider"
                )
                
                # Model selection
                yield Static("Model:", classes="select-text")
                yield Select(
                    options=model_options_for(self.provider),
                    allow_blank=False,
                    value=self._initial_model_value(),
                    id="ai-model"
                )
                
                # API Key input
                yield Static("API key:", classes="select-text")
                yield Input(
                    placeholder=f"Enter {PROVIDERS[self.provider]['label']} API key…",
                    value=mask_key(self._existing_key),
                    id="api-key-input"
                )
            
            with Center():
                yield HorizontalGroup(
                    Button("Apply", id="apply-ai-settings", variant="success"),
                    Button("Cancel (esc)", id="cancel-ai-settings", variant="default"),
                    id="ai-settings-actions"
                )

    def on_mount(self) -> None:
        self.provider_select = self.query_one("#ai-provider", Select)
        self.model_select = self.query_one("#ai-model", Select)
        self.key_input = self.query_one("#api-key-input", Input)

    def _initial_model_value(self) -> str:
        """Pick a valid initial model for the current provider."""
        opts = model_options_for(self.provider)
        values = {v for _, v in opts}
        if self.current_model in values:
            return self.current_model
        return opts[0][1] if opts else ""

    def _read_key(self, provider: str) -> str:
        return self.cfg.get(cfg_key_for(provider)) or ""

    def _write_key_if_changed(self, provider: str, typed_value: str):
        """
        If user typed a new key (non-empty and not just mask), save it.
        If left blank or mask, keep existing.
        """
        if not typed_value or typed_value == mask_key("x"):
            return  # keep existing
        self.cfg[cfg_key_for(provider)] = typed_value

    @on(Select.Changed, "#ai-provider")
    def on_provider_changed(self, event: Select.Changed) -> None:
        new_provider = str(event.value)
        if new_provider == self.provider:
            return
        
        # Update state
        self.provider = new_provider
        self._existing_key = self._read_key(self.provider)
        
        # Update model list + selection
        self.model_select.set_options(model_options_for(self.provider))
        self.current_model = self._initial_model_value()
        self.model_select.value = self.current_model
        
        # Update key field (mask if present, else empty placeholder)
        self.key_input.value = mask_key(self._existing_key)
        self.key_input.placeholder = f"Enter {PROVIDERS[self.provider]['label']} API key…"

    @on(Select.Changed, "#ai-model")
    def on_model_changed(self, event: Select.Changed) -> None:
        if self.current_model == str(event.value):
            return
        self.current_model = str(event.value)

    @on(Button.Pressed, "#apply-ai-settings")
    def on_apply_settings(self) -> None:
        typed = self.key_input.value.strip()

        # Save key if user actually typed one
        self._write_key_if_changed(self.provider, typed)

        # Always persist last used provider/model
        self.cfg["last_provider"] = self.provider
        self.cfg["last_model"] = self.current_model
        self.cfg.save()

        self.dismiss(True)

    @on(Button.Pressed, "#cancel-ai-settings")
    def on_cancel_settings(self) -> None:
        self.dismiss(None)

    def key_escape(self) -> None:
        self.dismiss(None)


## HelpModal

Interactive help screen accessible by pressing `?` in the main interface. Displays comprehensive documentation using Textual's MarkdownViewer component.

### Content Structure

The help documentation covers:

**Usage Instructions:** <br>
- How to start and stop recording <br>
- How the AI automatically detects edit commands <br>
- Example voice commands for editing <br>

**Settings Configuration:** <br>
- Audio settings: Model and language selection <br>
- AI settings: Provider and model configuration <br>

**Keyboard Shortcuts:** <br>
- Complete reference table of all available shortcuts <br>
- Quick access to common operations <br>

### Features

- **Markdown Rendering**: Full support for headings, lists, tables, and formatting <br>
- **Table of Contents**: Automatic navigation sidebar for long documents <br>
- **Scrollable**: Can handle extensive documentation <br>
- **Keyboard Dismissal**: Press Esc or click Close button to exit <br>

### Implementation

The `HELP_MARKDOWN` constant contains the documentation as a multi-line string. The MarkdownViewer widget handles parsing and rendering with syntax highlighting and proper formatting.

This approach allows easy updates to documentation without changing code structure - just edit the markdown content.

In [ ]:
#| export
#| include: false

HELP_MARKDOWN = """\
# TUI Writer

Help screen on how to use and customize TUI Writer.

## How to use

### Live transcriber with AI-powered editing:

- Press **SPACE** on your keyboard to start recording (top bar turns red)
- Simply start speaking and the transcriber will capture your words
- The AI automatically detects when you want to edit previous text
- Just say things like:
  - "Actually, change that to..."
  - "Delete the last sentence"
  - "Replace pizza with hamburgers"
- The AI will automatically apply edits and update the transcript
- Press **SPACE** again to stop recording

### Examples of voice commands:

**Adding new text:** Just speak normally, the AI will append it.

**Editing:** 
- "Change the word happy to excited"
- "Delete that last part"
- "Replace the first sentence with: Today was great"

## Settings & Configuration

### Audio Settings (Press 's'):
- Choose Whisper model (tiny to large-v3)
- Select transcription language
- Faster models = lower accuracy, slower models = better accuracy

### AI Model Settings (Press 'a'):
- Switch between OpenAI, Anthropic, and Google Gemini
- Choose specific models from each provider
- Configure API keys securely
- All providers work through the same unified interface

## Keyboard Shortcuts

| Key     | Action                    |
|---------|---------------------------|
| SPACE   | Start/Stop recording      |
| c       | Copy transcript           |
| s       | Audio settings            |
| a       | AI model settings         |
| ?       | This help screen          |
| q       | Quit application          |
"""

class HelpModal(ModalScreen):
    def compose(self) -> ComposeResult:
        with Container(id="help"):
            markdown_viewer = MarkdownViewer(HELP_MARKDOWN, show_table_of_contents=True)
            markdown_viewer.code_indent_guides = False
            yield markdown_viewer
            yield Center(Button("Close (esc)", variant="primary"))

    def on_button_pressed(self) -> None:
        self.app.pop_screen()

    def key_escape(self) -> None:
        self.app.pop_screen()


## Transcription TUI

The core application that integrates all components into a cohesive user experience.

### State Management

The application uses a simple state machine with two states:

- **IDLE**: Not recording, waiting for user input <br>
- **RECORDING**: Active transcription in progress <br>

The `state` attribute is reactive, meaning changes automatically trigger UI updates through the `watch_state()` method.

### Recording State Indicators

**Visual feedback when recording:**
- Header background turns red
- Title changes to "● RECORDING"
- Footer shows recording-specific shortcuts

**Visual feedback when idle:**
- Header background normal
- Title shows "○ STANDBY (provider/model)"
- Footer shows all available shortcuts

### AI Integration Strategy

The application maintains a single `TranscriptEditor` instance throughout its lifetime. This is important because:

1. **Memory Persistence**: The Chat inside TranscriptEditor maintains a `hist` (history) of all interactions
2. **Cross-Session Context**: Even after stopping and restarting recording, the AI remembers previous context
3. **Efficient Token Usage**: Conversation history allows contextual edits without re-sending entire transcript
4. **Better Edit Detection**: The AI understands references like "that" or "the last part" from earlier in the session

### Transcription Flow

1. **User presses SPACE** → `action_toggle_recording()` called
2. **State changes to RECORDING** → UI updates via `watch_state()`
3. **LiveTranscriber starts** → Begins capturing audio and running VAD
4. **Whisper transcribes utterances** → Calls `on_transcript_chunk()` with each result
5. **AI processes chunk** → `transcript_editor.process_chunk()` determines action
6. **Display updates**:
   - For appends: Add new line to Log widget
   - For edits: Clear Log and redisplay full transcript
7. **User presses SPACE again** → State returns to IDLE, transcriber stops

### Callback Pattern

The `on_transcript_chunk()` callback demonstrates graceful error handling:

```python
try:
    result = self.transcript_editor.process_chunk(text)
    # Update UI based on result
except Exception as e:
    # Fallback: just append text without AI processing
    self.transcript_display.write_line(text)
```

This ensures transcription continues even if AI processing fails (network issues, API limits, etc.).

### Configuration Loading

On mount, the application:
1. Loads configuration from disk
2. Reads API keys for all providers
3. Sets environment variables for LLM clients
4. Checks if configuration is valid
5. Displays appropriate warnings if setup is incomplete

### Keyboard Bindings

All keyboard shortcuts are declared in the `BINDINGS` class attribute:

- **q**: Quit application
- **?**: Open help modal
- **s**: Open audio settings
- **a**: Open AI model settings  
- **space**: Toggle recording on/off
- **c**: Copy transcript to clipboard

Textual automatically displays these in the footer and handles the key events.

In [ ]:
#| export
#| include: false

# Constants for transcriber configuration
DEFAULT_VAD_THRESHOLD = 0.5
DEFAULT_MIN_SPEECH_DURATION_MS = 250
DEFAULT_MIN_SILENCE_DURATION_MS = 500
MIN_EDIT_INSTRUCTION_WORDS = 2

class RecordingState(Enum):
    """High-level states the TUI can be in."""
    IDLE = auto()       # Not recording
    RECORDING = auto()  # Live transcription is running

class TranscriptionTUI(App):
    """Terminal user interface for live audio transcription and AI-powered editing.
    
    Features:
    - Live speech-to-text transcription using Whisper
    - AI-powered automatic edit detection and application
    - Configurable models and languages
    - Copy transcription to clipboard
    - AI model management (OpenAI/Anthropic/Gemini)
    """

    # Class attributes
    CSS = TEXTUAL_CSS
    AUTO_FOCUS = False
    state = reactive(RecordingState.IDLE)
    BINDINGS = [
        ("q", "quit", "Quit"),
        ("?", "help_modal", "Help"),
        ("s", "settings_modal", "Settings"),
        ("a", "ai_settings_modal", "AI Models"),
        ("space", "toggle_recording", "Start/Stop"),
        ("c", "copy_transcription", "Copy transcript")
    ]

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # Transcription state
        self._transcription_task: asyncio.Task | None = None
        self._transcriber: LiveTranscriber | None = None

        # Configuration
        self.whisper_model = "base"
        self.language = "en"
        self.transcribe_only = False
        
        # AI provider configuration
        self.ai_provider = ""
        self.ai_model = ""
        
        # Initialize transcript editor
        self.transcript_editor = None

    def compose(self) -> ComposeResult:
        """Construct and layout all UI widgets."""
        yield Header(show_clock=True)
        yield Log(id="transcript-display", auto_scroll=True)
        yield Footer()

    def _get_full_model_name(self) -> str:
        """Get the full model name in lisette format (provider/model)."""
        return f"{self.ai_provider}/{self.ai_model}"

    # === UI Actions (triggered by key bindings) ===
    def action_help_modal(self) -> None:
        self.push_screen(HelpModal())

    def action_settings_modal(self) -> None:
        self.push_screen(SettingsModal(self.whisper_model, self.language, self.transcribe_only), self.apply_settings)

    def action_ai_settings_modal(self) -> None:
        """Open AI model settings modal."""
        self.push_screen(
            AISettingsModal(), 
            self.apply_ai_settings
        ) 

    async def action_toggle_recording(self) -> None:
        """Spacebar: start if idle, stop if currently recording."""
        if self.state is RecordingState.IDLE:
            if self.transcribe_only:
                await self._start(self.on_transcribe_only)
            else:
                if not self.ai_model:
                    self.notify("Please setup AI model first. Press 'a' to set up", severity="warning")
                # Initialize transcript editor if needed
                if not self.transcript_editor:
                    self.transcript_editor = TranscriptEditor(self._get_full_model_name())
                await self._start(self.on_transcript_chunk)

            self.state = RecordingState.RECORDING
        elif self.state is RecordingState.RECORDING:
            self.title = "STOPPING..."
            await self._stop()
            self.state = RecordingState.IDLE

    def action_copy_transcription(self) -> None:
        if self.transcript_display.line_count > 0:
            text = "\n".join(self.transcript_display.lines)
            pyperclip.copy(text)
            self.notify("Copied transcription", title="Copied to clipboard!")
        else:
            self.notify("Nothing to copy yet!", severity="warning", title="Oops!")

    # === Settings & Modal Callbacks ===
    def apply_settings(self, settings: list[str]) -> None:
        """Apply Whisper model settings."""
        model, language, transcribe_only = settings
        self.whisper_model = model
        self.language = language
        self.transcribe_only = transcribe_only
    
    def apply_ai_settings(self, result) -> None:
        """Apply/update AI provider and model settings."""
        if result:
            self.cfg = get_cfg()
            self.load_all_api_keys()
            self.ai_provider = self.cfg["last_provider"]
            self.ai_model = self.cfg["last_model"]
            self.transcript_editor = TranscriptEditor(self._get_full_model_name())
            self.notify(
                f"Switched to {self.ai_provider.title()}: {self.ai_model}", 
                title="AI Provider Updated"
            )
            # Update the title to show the new model
            if self.state == RecordingState.IDLE:
                self.title = f"○ STANDBY ({self.ai_provider}/{self.ai_model})"

    # === Transcription Lifecycle ===
    async def _start(self, on_transcript_callback) -> None:
        self._transcriber = LiveTranscriber(
            model_id=self.whisper_model,
            language=self.language,
            on_transcript=on_transcript_callback,
            vad_threshold=DEFAULT_VAD_THRESHOLD,
            min_speech_duration_ms=DEFAULT_MIN_SPEECH_DURATION_MS,
            min_silence_duration_ms=DEFAULT_MIN_SILENCE_DURATION_MS,
        )
        self._transcription_task = asyncio.create_task(self._transcriber.start())

    async def _stop(self) -> None:
        try:
            if self._transcriber:
                await self._transcriber.stop()
            if self._transcription_task:
                await self._transcription_task
        finally:
            self._transcriber = None
            self._transcription_task = None

    # === Event Handlers & Callbacks ===
    def _normalize_text(self, text: str) -> str:
        """Normalize and clean text input."""
        return (text or "").strip()

    def on_transcribe_only(self, text: str) -> None:
        text = self._normalize_text(text)
        if not text:
            return
        self.transcript_display.write_line(text)


    async def on_transcript_chunk(self, text: str) -> None:
        """Called whenever the transcriber produces a new text chunk.
        Passes through AI to detect edits vs new text."""
        text = self._normalize_text(text)
        if not text:
            return
        
        try:
            # Process chunk through AI
            result = await self.transcript_editor.process_chunk(text)
            
            # Update display based on action
            if result["action"] == "edit":
                # Full transcript was edited - refresh entire display
                self.transcript_display.clear()
                self.transcript_display.write_lines(result["transcript"].splitlines(True))
                self.notify(f"✏️ Edit detected (tokens: {result['tokens_used']})", timeout=2)
            else:
                # New text appended - just add the line
                self.transcript_display.write_line(text)
            
        except Exception as e:
            # Fallback: just append the text without AI processing
            self.notify(f"AI processing failed: {str(e)}", severity="error", timeout=3)
            self.transcript_display.write_line(text)

    def watch_state(self, new_state: RecordingState) -> None:
        """Reactively update UI when recording state changes."""
        self.sub_title = ""

        match new_state:
            case RecordingState.IDLE:
                self.title = f"○ STANDBY ({self.ai_provider}/{self.ai_model})"
                self.header.remove_class("recording")
            case RecordingState.RECORDING:
                self.title = "● RECORDING"
                self.header.add_class("recording")

    def load_all_api_keys(self) -> None:
        if self.cfg.get("openai_key"):
            os.environ["OPENAI_API_KEY"] = self.cfg["openai_key"]
        
        if self.cfg.get("anthropic_key"):
            os.environ["ANTHROPIC_API_KEY"] = self.cfg["anthropic_key"]

        if self.cfg.get("gemini_key"):
            os.environ["GEMINI_API_KEY"] = self.cfg["gemini_key"]

    # === Lifecycle Hooks ===
    def on_mount(self) -> None:
        """Initialize widget references and set titles."""
        self.cfg = get_cfg()
        self.load_all_api_keys()
        if self.cfg["last_provider"] and self.cfg["last_model"]:
            self.ai_provider = self.cfg["last_provider"]
            self.ai_model = self.cfg["last_model"]

        # Check if provider is configured
        if not self.ai_provider:
            self.notify("No AI provider configured, press 'a' to set up.", severity="error")
            self.title = "○ STANDBY (No Provider Configured)"
        elif not self.cfg[f"{self.ai_provider}_key"]:
            self.notify("No API key recognized, press 'a' to set up.", severity="error")
            self.title = f"○ STANDBY ({self.ai_provider}/{self.ai_model})"
        else:
            self.notify(f"Using: {self.ai_provider} & {self.ai_model}. Loaded API Key successfully")
            self.title = f"○ STANDBY ({self.ai_provider}/{self.ai_model})"

        self.header = self.query_one(Header)
        self.transcript_display: Log = self.query_one("#transcript-display", Log)
        self.theme = "textual-dark"

    async def on_unmount(self) -> None:
        await self._stop()